# Insert triple for constituents related to objects

In [ ]:
query = """
            PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            PREFIX owl: <http://www.w3.org/2002/07/owl#>
            PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>

            SELECT ?event ?constituent ?object
            WHERE{
                ?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/ConstituentID> ?constituent .
                ?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/Prefix> ?event .
                ?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/RoleTypeID> ?RoleTypeID .
                ?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/ConXrefID> ?conxref .
                ?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/ID> ?ID .
                ?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/TableID> ?TableID .
                FILTER (?TableID = "108"^^<https://pressingmatter.nl/Bronbeek/ConXrefs/interger>) .
                ?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/RoleTypeID> <https://pressingmatter.nl/Bronbeek/RoleTypes/2> .
                ?object <https://pressingmatter.nl/Bronbeek/Objects/vocab/ObjectID> ?ID .
            }
        """

Run query:
```
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?constituent ?object
WHERE{
  	?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/ConstituentID> ?constituent .
  	?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/ConXrefID> ?conXref .
    ?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/ID> ?ID .
    ?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/TableID> ?TableID .
 	FILTER (?TableID = "108"^^<https://pressingmatter.nl/Bronbeek/ConXrefs/interger>) .
    {?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/RoleTypeID> <https://pressingmatter.nl/Bronbeek/RoleTypes/1>}
      UNION {?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/RoleTypeID> <https://pressingmatter.nl/Bronbeek/RoleTypes/2>} 
      UNION {?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/RoleTypeID> <https://pressingmatter.nl/Bronbeek/RoleTypes/5>} .
  	?object <https://pressingmatter.nl/Bronbeek/Objects/vocab/ObjectID> ?ID .
}
```

add triple

In [ ]:
import pandas

df = pandas.read_csv("conxref_object_link.csv").dropna(axis=1)
df.columns

In [ ]:
import rdflib, os
from rdflib import Graph, URIRef
from rdflib.namespace import SKOS

def insert_links(df):
    g = Graph()

    try:
        for _ , row in df.iterrows():
            conxrefs = URIRef(str(row[0]).strip().replace('"', ''))
            object = URIRef(str(row[1]).strip().replace('"', ''))
            g.add((conxrefs, SKOS.related, object))
        
    finally:
        print(len(g))
        return g

In [ ]:
insert_links(df).serialize(destination='conxreftoobject_graph.ttl')

# Insert triple for provenance activity (Acquisition)

run query:

```
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?event ?constituent ?object
WHERE{
  	?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/ConstituentID> ?constituent .
  	?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/Prefix> ?event .
  	?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/ConXrefID> ?conXref .
    ?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/ID> ?ID .
    ?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/TableID> ?TableID .
 	FILTER (?TableID = "108"^^<https://pressingmatter.nl/Bronbeek/ConXrefs/interger>) .
    ?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/RoleTypeID> <https://pressingmatter.nl/Bronbeek/RoleTypes/2> .
  	?object <https://pressingmatter.nl/Bronbeek/Objects/vocab/ObjectID> ?ID .
}
```

add triples

In [ ]:
import pandas

df = pandas.read_csv("provenance_activity.csv").dropna(axis=1)
df.columns

In [ ]:
from rdflib import Graph, URIRef, Namespace, Literal
from rdflib.namespace import SKOS, RDF, RDFS
from tqdm import tqdm 

def insert_links(df):
    crm = Namespace("http://www.cidoc-crm.org/cidoc-crm/")
    g = Graph()
    g.bind("crm",crm)

    try:
        for i , row in tqdm(df.iterrows()):

            event = str(row.iloc[0]).strip().replace('"', '')
            constituent = URIRef(str(row.iloc[1]).strip().replace('"', ''))
            object = URIRef(str(row.iloc[2]).strip().replace('"', ''))

            object_number = str(row['object']).split("/")[-1]

            acquisition_dict = {}
            acquisition_dict[object_number] = acquisition_dict.get(object_number, 0) + 1

            prov_activity = URIRef("www.example.com/Bronbeek/Provenance/"+str(i+1))
            acquisition = URIRef("www.example.com/Bronbeek/Acquisition/"+str(i+1))

            g.add((prov_activity, RDF.type, crm.E7_Activity))
            g.add((prov_activity, crm.P14_carried_out_by, constituent))
            g.add((prov_activity, crm.P2_has_type, URIRef(u"http://vocab.getty.edu/aat/300055863")))
            g.add((prov_activity, crm.P9_consists_of, acquisition))
            g.add((acquisition, RDF.type, crm.E8_Acquisition))
            g.add((acquisition, crm.P23_transferred_titled_from, constituent))
            g.add((acquisition, crm.P24_transferred_titled_of, object))
            g.add((acquisition, RDFS.label, Literal(event)))
            
        
    finally:
        print(len(g))
        g.add((URIRef(u"http://vocab.getty.edu/aat/300055863"), RDF.type, crm.E55_Type))
        g.add((URIRef(u"http://vocab.getty.edu/aat/300055863"), RDFS.label, Literal("acquisition_event.ttl")))
        return g

In [ ]:
insert_links(df).serialize(destination='provenance_activity.ttl')

# Add triple for former owner

Run query:
```
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?event ?constituent ?object
WHERE{
  	?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/ConstituentID> ?constituent .
  	?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/Prefix> ?event .
  	?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/ConXrefID> ?conXref .
    ?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/ID> ?ID .
    ?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/TableID> ?TableID .
 	FILTER (?TableID = "108"^^<https://pressingmatter.nl/Bronbeek/ConXrefs/interger>) .
    ?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/RoleTypeID> <https://pressingmatter.nl/Bronbeek/RoleTypes/5> .
  	?object <https://pressingmatter.nl/Bronbeek/Objects/vocab/ObjectID> ?ID .
}
```

In [ ]:
import pandas

df = pandas.read_csv("former_owner.csv").dropna(axis=1)
df.columns

In [ ]:
from rdflib import Graph, URIRef, Namespace, Literal
from rdflib.namespace import SKOS, RDF, RDFS
from tqdm import tqdm 

def insert_links(df):
    g = Graph()
    crm = Namespace("http://www.cidoc-crm.org/cidoc-crm/")
    g.bind("crm",crm)

    try:
        for i , row in tqdm(df.iterrows()):
            constituent = URIRef(str(row[1]).strip().replace('"', ''))
            object = URIRef(str(row[2]).strip().replace('"', ''))

            g.add((object, crm.P51_has_former_or_current_owner, constituent)) 
    finally:
        print(len(g))
        return g

In [ ]:
insert_links(df).serialize(destination='former_owner.ttl')

# Add triple for person related to object (RoleTypeID=1)

Run query:
```
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?event ?constituent ?object
WHERE{
  	?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/ConstituentID> ?constituent .
  	?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/Prefix> ?event .
  	?conxrefdetailID <https://pressingmatter.nl/Bronbeek/ConXrefDetails/vocab/ConXrefID> ?conXref .
    ?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/ID> ?ID .
    ?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/TableID> ?TableID .
 	FILTER (?TableID = "108"^^<https://pressingmatter.nl/Bronbeek/ConXrefs/interger>) .
    ?conXref <https://pressingmatter.nl/Bronbeek/ConXrefs/vocab/RoleTypeID> <https://pressingmatter.nl/Bronbeek/RoleTypes/5> .
  	?object <https://pressingmatter.nl/Bronbeek/Objects/vocab/ObjectID> ?ID .
}
```

In [ ]:
import pandas

df = pandas.read_csv("person_related.csv").dropna(axis=1)
df.columns

In [ ]:
from rdflib import Graph, URIRef, Namespace, Literal
from rdflib.namespace import SKOS, RDF, RDFS
from tqdm import tqdm 

def insert_links(df):
    g = Graph()
    crm = Namespace("http://www.cidoc-crm.org/cidoc-crm/")
    g.bind("crm",crm)

    try:
        for i , row in tqdm(df.iterrows()):
            constituent = URIRef(str(row[0]).strip().replace('"', ''))
            object = URIRef(str(row[1]).strip().replace('"', ''))

            g.add((object, SKOS.related, constituent)) 
    finally:
        print(len(g))
        return g

In [ ]:
insert_links(df).serialize(destination='person_related.ttl')

# Insert provenance activity based on accession lot

In [ ]:
from rdflib import URIRef, Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
from tqdm import tqdm
import uuid
import os

def insert_links(df):
    crm = Namespace("http://www.cidoc-crm.org/cidoc-crm/")
    g = Graph()
    g.bind("crm", crm)

    try:
        for _, row in tqdm(df.iterrows()):
            
            object = URIRef(str(row.iloc[0]).strip().replace('"', ''))
            accession = URIRef(str(row.iloc[1]).strip().replace('"', ''))
            status = str(row.iloc[2]).strip().replace('"', '')
            method = str(row.iloc[3]).strip().replace('"', '')
            
            time = str(row.iloc[4]).strip().replace('"', '') if not row.iloc[4]=="" else None

            object_number = str(object).split("/")[-1]
            
            acquisition_dict = {}
            acquisition_dict[object_number] = acquisition_dict.get(object_number, 0) + 1

            
            prov_activity = URIRef("https://www.pressingmatter.nl/Bronbeek/Provenance/" + str(object_number))
            acquisition = URIRef("https://www.pressingmatter.nl/Bronbeek/Acquisition/2/" + str(object_number) + "/" + str(acquisition_dict[object_number]))

            g.add((prov_activity, RDF.type, crm.E7_Activity))
            g.add((prov_activity, RDFS.label, Literal(status)))
            g.add((prov_activity, crm.P2_has_type, URIRef(u"http://vocab.getty.edu/aat/300055863")))
            g.add((prov_activity, crm.P9_consists_of, acquisition))
            g.add((acquisition, RDF.type, crm.E8_Acquisition))
            g.add((acquisition, crm.P24_transferred_title_of, object))
            g.add((acquisition, RDFS.label, Literal(method)))
            
            if time: 
                time_BNode = URIRef("https://www.pressingmatter.nl/Bronbeek/time/" + str(uuid.uuid1()))
                g.add((acquisition, crm.P4_has-time, time_BNode))
                g.add((time_BNode, RDF.type, crm.E52_Time))
                g.add((time_BNode, crm.P82a_begin_of_the_begin, Literal(time))) 
                g.add((time_BNode, crm.P82b_end_of_the_end, Literal(time)))
                
    finally:
        print(len(g))
        g.add((URIRef(u"http://vocab.getty.edu/aat/300055863"), RDF.type, crm.E55_Type))
        g.add((URIRef(u"http://vocab.getty.edu/aat/300055863"), RDFS.label, Literal("Provenance Activity")))
        return g

In [ ]:
import pandas
df = pandas.read_csv("../Bronbeek_Data/LD-unzipped/queryResults.csv").dropna(axis=1)

In [ ]:
insert_links(df).serialize(destination=os.path.join("../Bronbeek_Data/LD-unzipped/", 'provenance_activity.ttl'))